### This is the submission that got me 7th place. It's not very readable and there are a lot of things to be improved so sorry about that, but I've been told to share it as soon as I could. This is the version using future information. See [link to practical version as soon as I finish it] for the simpler, leak-free version.
### The main ideas can be found here: https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/276506#1534495

#### Import libraries, input paths etc.

In [ ]:
from IPython.core.display import display, HTML
import glob

path_submissions = '/'
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'
all_train_book = glob.glob(f'/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/**/*.parquet')
all_test_book = glob.glob(f'/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/**/*.parquet')

import pandas as pd
import numpy as np
import os
import gc

from math import sqrt
from joblib import Parallel, delayed
from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.preprocessing import minmax_scale
from sklearn.neighbors import NearestNeighbors
import lightgbm as lgb
from sklearn.cluster import KMeans
from tensorflow.keras.backend import sigmoid
from itertools import chain
from itertools import permutations
from math import floor

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation

import random
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
from tensorflow.keras import backend as K
from scipy.linalg import lstsq
import seaborn as sns

#### Function to retrieve an array of usable time_ids.

In [ ]:
def get_time_ids(is_train = True):
    files = all_train_book if is_train else all_test_book
    return np.sort(pd.concat([pd.read_parquet(file, columns = ['time_id']) for file in files], ignore_index = True)['time_id'].unique())

#### List to split each time_id.

In [ ]:
time_splits = [0, 100, 200, 300, 400, 500, 600]

#### Account for the test dataset potentially not having enough time_ids.

In [ ]:
test_time_id_len = len(get_time_ids(False))

is_dummy = test_time_id_len == 1
prox = test_time_id_len if test_time_id_len in range(2,6) else 6

if test_time_id_len == 1:
    unord_group_size = [20, 50, 100]
elif test_time_id_len in range(2,7):
    unord_group_size = []
elif test_time_id_len in range(7,24):
    unord_group_size = [test_time_id_len]
elif test_time_id_len in range(24,48):
    unord_group_size = [round(test_time_id_len/2), test_time_id_len]
elif test_time_id_len in range(48, 100):
    unord_group_size = [round(test_time_id_len/4), round(test_time_id_len/2), test_time_id_len]
else:
    unord_group_size = [20, 50, 100]

#### Define various functions for feature engineering.

In [ ]:
def log_return(series):
    return np.log(series).diff()

# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# RQQ replaced rolling window with convolve for performance
def realized_quadpower_quarticity(series):
    convolved = np.exp(np.convolve(np.log(series.to_numpy()), [1, 1, 1, 1], mode='valid'))
    return (np.sum(convolved) * (convolved.shape[0]+3) * (np.pi**2))/4

def tendency(price, vol):    
    df_diff = np.diff(price.to_numpy())
    val = (df_diff/price.to_numpy()[1:])*100
    power = np.sum(val*vol.to_numpy()[1:])
    return(power)

def iqr(series): 
    return np.percentile(series.to_numpy(),75) - np.percentile(series.to_numpy(),25)

def abs_diff(series): 
    return np.median(np.abs(series.to_numpy() - np.mean(series.to_numpy())))

def energy(series): 
    return np.mean(series.to_numpy()**2)

def f_max(series): 
    return np.sum(series.to_numpy() > np.mean(series.to_numpy()))

def f_min(series): 
    return np.sum(series.to_numpy() < np.mean(series.to_numpy()))

def df_max(series): 
    return np.sum(np.diff(series.to_numpy()) > 0)

def df_min(series): 
    return np.sum(np.diff(series.to_numpy()) < 0)

#### Fix broken offsets (in public test dataset)

In [ ]:
def list_names(dic, prefix = '', splits = []):
    return list(chain(*[[f'{prefix}{key}_{func.__name__}{t}' for func in lis] for key, lis in dic.items() for t in ['']+splits]))

def fix_offsets(book_df, trade_df):
    offsets = book_df.groupby(['time_id']).agg({'seconds_in_bucket':'min'})
    offsets.columns = ['offset']
    book_df = book_df.join(offsets, on='time_id')
    trade_df = trade_df.join(offsets, on='time_id')
    book_df.seconds_in_bucket = book_df.seconds_in_bucket - book_df.offset
    trade_df.seconds_in_bucket = trade_df.seconds_in_bucket - trade_df.offset
    book_df.drop(columns=['offset'], inplace=True)
    trade_df.drop(columns=['offset'], inplace=True)
    return book_df, trade_df

def fix_offsets_book(data_df):
    offsets = data_df.groupby(['time_id']).agg({'seconds_in_bucket':'min'})
    offsets.columns = ['offset']
    data_df = data_df.join(offsets, on='time_id')
    data_df.seconds_in_bucket = data_df.seconds_in_bucket - data_df.offset
    return data_df

def ffill(data_df):
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], np.arange(0,600)], names = ['time_id', 'seconds_in_bucket']), method='ffill')
    return data_df.reset_index()

#### De-normalize stock prices using tick sizes, inspired by [this](https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/256725#1449558) code snippet by @stassi. Generate arrays for the beginning, average and end prices of each time_id.

In [ ]:
def calc_price(df):
    book = df.to_numpy()a
    diff = np.diff(book.transpose().flatten())
    mindiff = np.nanmin(abs(diff[diff != 0]))
    return book[:60, [0, 2]].mean() * 0.01 / mindiff, book[:, [0, 2]].mean() * 0.01 / mindiff, book[-60:, [0, 2]].mean() * 0.01 / mindiff

def calc_prices(r, is_start = True):
    df = ffill(fix_offsets_book(pd.read_parquet(r.book_path)))
    df = df[['time_id', 'ask_price1', 'ask_price2', 'bid_price1', 'bid_price2']]
    df = df.set_index('time_id')
    df = df.groupby(level='time_id').apply(calc_price).to_frame('price').reset_index()
    df['stock_id'] = r.stock_id
    return df

def calc_price_2(df):
    book = df.to_numpy()
    diff = np.diff(book.transpose().flatten())
    mindiff = np.nanmin(abs(diff[diff != 0]))
    return book[0, [0, 2]].mean() * 0.01 / mindiff, book[-1, [0, 2]].mean() * 0.01 / mindiff

In [ ]:
def get_book_files(is_train = True):
    df_files = pd.DataFrame(
        {'book_path': all_train_book if is_train else all_test_book}) \
        .eval('stock_id = book_path.str.extract("stock_id=(\d+)").astype("int")', engine='python')
    df_prices = pd.concat(Parallel(n_jobs=4, verbose=51)(delayed(calc_prices)(r) for _, r in df_files.iterrows()))
    return df_prices.pivot(index = 'time_id', columns = 'stock_id', values = 'price')

def get_price_dfs(is_train = True):
    df_prices = get_book_files(is_train)
    df_prices = df_prices.dropna(axis = 1)
    arr = np.array([[list(x) for x in time_id] for time_id in df_prices.to_numpy()])
    arr = np.array([(arr[:, i, :] - arr[:, i, :].min())/(arr[:, i, :].max() - arr[:, i, :].min()) for i in range(arr.shape[1])])
    np_start = arr[..., 0].transpose()
    np_avg = arr[..., 1].transpose()
    np_end = arr[..., 2].transpose()
    df_start = pd.DataFrame(np_start, index = df_prices.index, columns = df_prices.columns)
    df_avg = pd.DataFrame(np_avg, index = df_prices.index, columns = df_prices.columns)
    df_end = pd.DataFrame(np_end, index = df_prices.index, columns = df_prices.columns)
    return df_start, df_avg, df_end

In [ ]:
def get_prices(stock_id, list_time_ids, is_train = True):
    df_path = f'{data_dir}book_{"train" if is_train else "test"}.parquet/stock_id={str(stock_id)}/'
    df = pd.read_parquet(df_path, columns = ['time_id', 'ask_price1', 'ask_price2', 'bid_price1', 'bid_price2'])
    df = df.set_index('time_id')
    df = df.groupby(level='time_id').apply(calc_price_2).to_frame('price')
    df = df.reindex(list_time_ids)
    df = df.applymap(lambda x: (-1,-1) if x is np.nan else x)
    arrr = df['price']
    np_start = np.array([val[0] for val in arrr])
    np_end = np.array([val[1] for val in arrr])
    return np_start, np_end

#### All of the cluster stuff was added there as a botched way to make this work with datasets of different sizes. There is obviously a much better way to do it (like using something like $\sqrt{\sum{}{}(\log{(\frac{u_i}{v_i})})^2}$ instead of Eucledean distances between $u_i$ and $v_i$ for NN distances) but unfortunately I've only thought of that after the submission deadline. This also accounted for the potential presence of filler data in the dataset.


In [ ]:
def get_neighbors(df_avg, is_train = True):
    np_avg = df_avg.to_numpy()
    if len(df_avg) == 1:
        n_nei = 1
    elif unord_group_size:
        n_nei = unord_group_size[-1]
    else:
        n_nei = prox
    if is_train:
        n_clusters = 12
        cluster_list, nbrs_list, distances, indices = [], [], [], []
        kmeans = KMeans(n_clusters = n_clusters, random_state = 0).fit(df_avg.to_numpy())
        for i in range(n_clusters):
            cluster_ind = np.arange(np_avg.shape[0])[kmeans.labels_ == i]
            cluster_list.append(cluster_ind)
            nbrs_list.append(NearestNeighbors(n_neighbors=n_nei+30).fit(np_avg[cluster_ind]))
        for i, vector in enumerate(np_avg):
            curr_group = kmeans.labels_[i]
            dist, indi = nbrs_list[curr_group].kneighbors([vector])
            dist, indi = dist[0], indi[0]
            to_delete = set(random.sample(range(1,len(indi)), 30))
            dist = np.array([d for i,d in enumerate(dist) if i not in to_delete])
            indi = np.array([d for i,d in enumerate(indi) if i not in to_delete])
            distances.append(dist)
            indices.append(cluster_list[curr_group][indi])
        return np.array(distances), np.array(indices)
    else: 
        avg_nbrs = NearestNeighbors(n_neighbors=n_nei).fit(np_avg)
        return avg_nbrs.kneighbors(np_avg)

#### Brute-force the shortest Hamilton path between the nearest time_ids in an attempt to re-order them: 

#### Take arrays of start and end prices of each time_id, and re-order them to minimise the Euclidean distance.

In [ ]:
def get_ordered_prox(df_start, df_end, indices):
    np_start = df_start.to_numpy()
    np_end = df_end.to_numpy()
    def vec_distance(vec1, vec2):
        return np.sqrt(np.sum((vec1 - vec2)**2))
    def get_total_distance(seq):
        return sum([vec_distance(np_start[i], np_end[j]) for i,j in zip(seq[1:], seq[:-1])])
    indices = np.array([list(list(permutations(a))[np.argmin(list(map(get_total_distance, permutations(a))))]) for a in indices])
    distances = np.array([list(chain(*[[vec_distance(np_start[j], np_end[j]), vec_distance(np_start[i], np_end[j])] for i, j in zip(seq[1:], seq[:-1])] \
                                + [[vec_distance(np_start[seq[-1]], np_end[seq[-1]])]])) for seq in indices])
    return distances, indices

#### Function to get the mean, trend and error of an array using least squares. This is used to process features generated in time windows to reduce dimensionality and improve interpretability of feature importance.

In [ ]:
def get_dir_stats(arr):
    coefs, err, _, _ = lstsq(np.array([np.ones(len(arr)), np.arange(len(arr))]).transpose(), arr)
    return np.array([coefs[0] + len(arr)*coefs[1], coefs[1], err])

#### Get stats from the nearest time_ids.

In [ ]:
def get_prox_stats(df, ord_distances, ord_indices, start_prices, end_prices):
    # INDEX IN ORD
    final_df = pd.DataFrame()
    ind = np.array([np.where(elem == i)[0][0] for i, elem in enumerate(ord_indices)])
    # USED COLS / FUNCS
    next_group_list = ['log_return1_realized_volatility_intercept', 
                       'total_volume_sum', 
                       'trade_order_count_sum_intercept',
                       'trade_order_count_sum_slope',
                       'trade_size_sum_intercept',
                       'trade_size_sum_slope',
                       'trade_size_sum_error',
                       'log_return1_realized_volatility_slope', 
                       'log_return1_realized_volatility_error']
    prox_group_list = ['log_return1_realized_volatility_intercept', 
                       'log_return1_realized_volatility_slope', 
                       'log_return1_realized_volatility_error', 
                       'trade_order_count_sum_intercept']
    func_list = [np.mean, np.std, np.min, np.max]
    # RETURN 0 IF ONLY 1 COL
    if len(ord_indices[0]) == 1:
        name_list = [f'prox_{col}_{func.__name__}' for col in prox_group_list for func in func_list] + \
        [f'next_{col}' for col in next_group_list] + \
        [f'dist_{func.__name__}' for func in func_list] + \
        ['dist_window_rv', 'dist_wlr_mean', 'dist_wlr_std', 'dist_wlr_amin', 'dist_wlr_amax', 'dist_to_next', 'next_price_diff', 'ind']
        return pd.DataFrame(data = np.zeros((len(ord_indices[0]), len(name_list))), columns = name_list)
    # PROX AND NEXT
    for col in next_group_list:
        final_df[f'next_{col}'] = np.array([df[col][smth[curr+1]] if curr < prox-1 else -1 for curr, smth in zip(ind, ord_indices)])
    for col in prox_group_list:
        for func in func_list:
            final_df[f'prox_{col}_{func.__name__}'] = func(np.array([df.iloc[ord_indices[index]][col] for index in df.index]), axis = 1)
    # DISTANCES
    window_log_return = [pd.Series(chain(*[[start_prices[t_id], end_prices[t_id]] for t_id in ord_indices[index]])) for index in df.index]
    window_log_return = [log_return(p_list)[1:] for p_list in window_log_return]
    final_df['dist_window_rv'] = np.array([realized_volatility(wlr) for wlr in window_log_return])
    final_df['dist_wlr_std'] = np.array([wlr.std() if wlr.isnull().values.any() == False else -1 for wlr in window_log_return])
    for func in [np.mean, np.min, np.max]:
        final_df[f'dist_wlr_{func.__name__}'] = func(window_log_return, axis = 1)
    final_df['dist_to_next'] = np.array([distance[curr * 2 + 1] if curr < prox-1 else -1 for distance, curr in zip(ord_distances, ind)])
    for func in func_list:
        final_df[f'dist_{func.__name__}'] = func(ord_distances, axis = 1)
    final_df['ind'] = ind
    price_diff_next = []
    for end_price, curr, smth in zip(end_prices, ind, ord_indices):
        if curr >= prox-1 or start_prices[smth[curr+1]] == -1 or end_prices[smth[curr]] == -1:
            price_diff_next.append(-1)
        else:
            price_diff_next.append(abs(np.log(end_price / start_prices[smth[curr+1]])))
    final_df['next_price_diff'] = np.array(price_diff_next)
    return final_df

#### Get the n closest time_ids.

In [ ]:
def get_unord_prox(df, distances, indices, prices, count, dum):
    cols = ['log_return1_realized_volatility_intercept', 'total_volume_sum', 'trade_log_return_realized_volatility_intercept']
    func_list = [np.mean, np.std, np.min, np.max]
    func_list_dist = [np.mean, np.std, np.min, np.max] if count == unord_group_size[0] else [np.mean, np.std, np.max]
    final_df = pd.DataFrame()
    if count == unord_group_size[0]:
        final_df[f'prox_{count}_price_min_max'] = [0] if dum else np.array([prices[ind].min()/prices[ind].max() for ind in indices])
        final_df[f'prox_{count}_price_mean_max'] = [0] if dum else np.array([prices[ind].mean()/prices[ind].max() for ind in indices])
    for func in func_list:
        for col in cols:
            final_df[f'prox_{count}_{col}_{func.__name__}'] = [0] if dum else func(np.array([df.iloc[indices[index]][col] for index in df.index]), axis = 1)
    for func in func_list_dist:
        final_df[f'dist_{count}_{func.__name__}'] = [0] if dum else func(distances[...,1:], axis = 1)
    return final_df

#### Process book data (through combinations of the above functions)

In [ ]:
# Function to preprocess book data (for each stock id)
def book_preprocessor(df, stock_id, time_ids):
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.mean, realized_quadpower_quarticity, count_unique],
        'wap2': [np.mean, realized_quadpower_quarticity, count_unique],
        'log_return1': [np.sum, np.min, np.max],
        'log_return2': [np.sum],
        'wap_balance': [np.mean, np.max],
        'price_spread':[np.mean, np.max],
        'price_spread2':[np.mean, np.max],
        'bid_spread':[np.mean, np.max],
        'ask_spread':[np.mean, np.max],
        'total_volume':[np.sum],
        'volume_imbalance':[np.mean],
        "bid_ask_spread":[np.mean, np.max],
    }
    
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
    }
    
    df_feature = df.groupby(['time_id']).agg(create_feature_dict).reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    df_feature = df_feature.set_index('time_id_').reindex(pd.Series(time_ids))
    df_features = []
    for start, stop in zip(time_splits[:-1], time_splits[1:]):
        df_tmpp = df[(df['seconds_in_bucket'] >= start) & (df['seconds_in_bucket'] < stop)].groupby(['time_id']).agg(create_feature_dict_time).reset_index()
        df_tmpp.columns = ['_'.join(col) for col in df_tmpp.columns]
        df_tmpp = df_tmpp.set_index('time_id_').reindex(pd.Series(time_ids))
        df_tmpp = df_tmpp.fillna(0)
        df_features.append(df_tmpp[list_names(create_feature_dict_time)].to_numpy().transpose())
    df_features = np.stack(df_features, axis = 2)
    df_features = np.apply_along_axis(get_dir_stats, 2, df_features)
    for name, vals in zip(list_names(create_feature_dict_time), df_features):
        for attr, col in zip(['intercept', 'slope', 'error'], vals.transpose()):
            df_feature[f'{name}_{attr}'] = col
    
    # Create row_id so we can merge
    df_feature.reset_index(inplace = True)
    df_feature['row_id'] = df_feature['index'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['index'], axis = 1, inplace = True)
    return df_feature

#### Process trade data.

In [ ]:
# Function to preprocess trade data (for each stock id)
def trade_preprocessor(df, stock_id, time_ids):
    #df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
    # Dict for aggregations
    create_feature_dict = {
        'amount':[np.min],
        'price':[abs_diff, energy, iqr]
    }
    
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'order_count':[np.sum],
        'size':[np.sum]
    }
    
    df_feature = df.groupby(['time_id']).agg(create_feature_dict).reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    df_feature = df_feature.set_index('time_id_').reindex(pd.Series(time_ids))
    df_features = []
    for start, stop in zip(time_splits[:-1], time_splits[1:]):
        df_tmpp = df[(df['seconds_in_bucket'] >= start) & (df['seconds_in_bucket'] < stop)].groupby(['time_id']).agg(create_feature_dict_time).reset_index()
        df_tmpp.columns = ['_'.join(col) for col in df_tmpp.columns]
        df_tmpp = df_tmpp.set_index('time_id_').reindex(pd.Series(time_ids))
        df_tmpp = df_tmpp.fillna(0)
        df_features.append(df_tmpp[list_names(create_feature_dict_time)].to_numpy().transpose())
    df_features = np.stack(df_features, axis = 2)
    df_features = np.apply_along_axis(get_dir_stats, 2, df_features)
    for name, vals in zip(list_names(create_feature_dict_time), df_features):
        for attr, col in zip(['intercept', 'slope', 'error'], vals.transpose()):
            df_feature[f'{name}_{attr}'] = col
    df_feature.reset_index(inplace = True)

    # Get the stats for different windows
    df_feature = df_feature.add_prefix('trade_')
    df_feature['row_id'] = df_feature['trade_index'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_index'], axis = 1, inplace = True)
    return df_feature

In [ ]:
trivial_nan_once = {
    'amount':[np.min],
    'price':[iqr]
}

#### Get time_id aggregated stats.

In [ ]:
def get_time_stats(df):
    vol_cols = [f'log_return1_realized_volatility_{t}' for t in ['intercept', 'slope', 'error']]
    vol_cols += ['trade_order_count_sum_intercept', 'next_trade_order_count_sum_intercept', 'trade_order_count_sum_slope', 'next_trade_order_count_sum_slope']
    vol_cols += [f'next_log_return1_realized_volatility_{t}' for t in ['intercept', 'slope', 'error']]
    vol_cols += [f'prox_log_return1_realized_volatility_intercept_{func}' for func in ['mean', 'std', 'amin', 'amax']]
    vol_cols += [f'prox_{num}_log_return1_realized_volatility_intercept_{func}' for num in unord_group_size for func in ['mean', 'std']]
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['time_id__time'], axis = 1, inplace = True)
    return df

#### Combine all above function.

In [ ]:
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    # Parrallel for loop
    df_start, df_avg, df_end = get_price_dfs(is_train)
    unord_dist, unord_ind = get_neighbors(df_avg, is_train)
    distances, indices = get_ordered_prox(df_start, df_end, unord_ind[...,:prox])
    list_time_ids = get_time_ids(is_train)
    def for_joblib(stock_id):
        np_start, np_end = get_prices(stock_id, list_time_ids, is_train)
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
            
        book_df, trade_df = fix_offsets(pd.read_parquet(file_path_book), pd.read_parquet(file_path_trade))
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(book_df, stock_id, list_time_ids), trade_preprocessor(trade_df, stock_id, list_time_ids), on = 'row_id', how = 'left')
        colls = list_names(trivial_nan_once, 'trade_')
        for col in colls:
            df_tmp[col] = df_tmp[col].fillna(0)
        # Return the merge dataframe
        df_tmp = df_tmp.join(get_prox_stats(df_tmp, distances, indices, np_start, np_end))
        for i in unord_group_size:
            df_tmp = df_tmp.join(get_unord_prox(df_tmp, unord_dist[...,:i], unord_ind[...,:i], np_start, i, is_dummy and not is_train))
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 51)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

#### Drop unnecessary features.

In [ ]:
def generate_df(is_train):
    df = pd.read_csv(f'{data_dir}{"train" if is_train else "test"}.csv')
    df['row_id'] = df['stock_id'].astype(str) + '-' + df['time_id'].astype(str)
    print(f'Our {"training" if is_train else "testing"} set has {df.shape[0]} rows')
    stock_ids = df['stock_id'].unique()
    # Preprocess them using Parallel and our single stock id functions
    pre_ = preprocessor(stock_ids, is_train = is_train)
    df = df.merge(pre_, on = ['row_id'], how = 'left')
    df = get_time_stats(df)
    df.drop(columns = [f'prox_trade_order_count_sum_intercept_{func}' for func in ['amax', 'std']], inplace = True)
    df.drop(columns = ['prox_log_return1_realized_volatility_slope_amin'])
    df.drop(columns = [f'trade_size_sum_{att}' for att in ['error', 'slope']], inplace = True)
    if len(unord_group_size) == 3:
        for grp in unord_group_size:
            if grp != unord_group_size[1]:
                df.drop(columns = [f'prox_{grp}_total_volume_sum_std'], inplace = True)
            if grp == unord_group_size[0]:
                df.drop(columns = [f'prox_{grp}_trade_log_return_realized_volatility_intercept_std'], inplace = True)
                df.drop(columns = [f'prox_{grp}_log_return1_realized_volatility_intercept_amax'], inplace = True)
            if grp == unord_group_size[2]:
                df.drop(columns = [f'prox_{grp}_total_volume_sum_amax'], inplace = True)
    return df

#### Generate train and test DFs.

In [ ]:
test = generate_df(False)

In [ ]:
train = generate_df(True)

In [ ]:
col_names = [col for col in list(train.columns)
            if col not in {"stock_id", "time_id", "target", "row_id"}]
cflen = len(col_names)

#### Print correlation heatmap.

In [ ]:
corr = train[col_names].corr()
mask = np.triu(corr)
sns.heatmap(corr, xticklabels = corr.columns, yticklabels = corr.columns, annot = True, ax = plt.subplots(figsize = (200,100))[1], mask = mask)

#### Define LGBM model.

In [ ]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
seed0 = 2021
params0 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':100,
    'min_data_in_leaf':500,
    'learning_rate': 0.05,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.5,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}

In [ ]:
def train_and_evaluate_lgb(train, test, params):
    # Hyperparammeters (just basic)
    
    features = [col for col in train.columns if col not in {"time_id", "target", "row_id", "stock_id"}]
    y = train['target']
    # Create out of folds array
    oof_predictions = np.zeros(train.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = 2021, shuffle = True)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train[features], y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val[features], y_val, weight = val_weights)
        model = lgb.train(params = params,
                          num_boost_round=1000,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=50,
                          feval = feval_rmspe)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val[features])
        # Predict the test set
        test_predictions += model.predict(test[features]) / 5
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    lgb.plot_importance(model, figsize = (30, 160))
    # Return test predictions
    return test_predictions
# Traing and evaluate
predictions_lgb = train_and_evaluate_lgb(train, test, params0)

#### Define FFNN model.

In [ ]:
def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss', patience = 20, verbose=0,
    mode = 'min', restore_best_weights = True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss', factor = 0.2, patience = 7, verbose = 0,
    mode = 'min')

In [ ]:
# kfold based on the knn++ algorithm

out_train = pd.read_csv(f'{data_dir}train.csv')
out_train = out_train.pivot(index = 'time_id', columns = 'stock_id', values = 'target')

out_train = out_train.fillna(out_train.mean())
out_train.head()

# code to add the just the read data after first execution
# data separation based on knn ++
n_folds = 5 # number of folds
index = []
totDist = []
values = []
# generates a matriz with the values of 
mat = out_train.to_numpy()
scaler = MinMaxScaler(feature_range = (-1, 1))
mat = scaler.fit_transform(mat)
nind = int(mat.shape[0]/n_folds) # number of individuals

# adds index in the last column
mat = np.c_[mat,np.arange(mat.shape[0])]
lineNumber = np.random.choice(np.array(mat.shape[0]), size=n_folds, replace=False)
lineNumber = np.sort(lineNumber)[::-1]

for n in range(n_folds):
    totDist.append(np.zeros(mat.shape[0]-n_folds))

# saves index
for n in range(n_folds):  
    values.append([lineNumber[n]])    

s = []
for n in range(n_folds):
    s.append(mat[lineNumber[n],:])
    mat = np.delete(mat, obj = lineNumber[n], axis=0)

for n in range(nind - 1):    
    luck = np.random.uniform(0,1,n_folds) 
    for cycle in range(n_folds):
         # saves the values of index           
        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)
        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities

        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn += 1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(n_folds):
            totDist[n_iter] = np.delete(totDist[n_iter],obj = lineNumber[cycle], axis = 0)
            j = 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis = 0)

for n_mod in range(n_folds):
    values[n_mod] = out_train.index[values[n_mod]]

In [ ]:
train.drop(columns = ['row_id'], inplace = True)
train.replace([np.inf, -np.inf], np.nan, inplace = True)
test.replace([np.inf, -np.inf], np.nan, inplace = True)
for col in col_names:
    qt = QuantileTransformer(random_state = 21, n_quantiles = 2000, output_distribution = 'normal')
    train[col] = qt.fit_transform(train[[col]])
    test[col] = qt.transform(test[[col]])

In [ ]:
#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

get_custom_objects().update({'swish': Activation(swish)})

def base_model():
    inputs = keras.Input(shape=(cflen,), name = 'num_data')
    x = keras.layers.Dense(128, activation = 'swish')(inputs)
    x = keras.layers.Dense(64, activation = 'swish')(x)
    x = keras.layers.Dense(32, activation = 'swish')(x)
    outputs = keras.layers.Dense(1, activation = 'linear', name = 'prediction')(x)
    model = keras.Model(
        inputs = inputs,
        outputs = outputs,
    )
    return model

In [ ]:
scores_folds = {}
model_name = 'NN'
pred_name = f'pred_{model_name}'

kf = model_selection.KFold(n_splits = n_folds, shuffle=True, random_state=2020)
scores_folds[model_name] = []
counter = 1

features_to_consider = col_names

train[features_to_consider] = train[features_to_consider].fillna(train[features_to_consider].mean())
test[features_to_consider] = test[features_to_consider].fillna(train[features_to_consider].mean())

train[pred_name] = 0
test['target'] = 0
test_predictions_nn = np.zeros(test.shape[0])

for n_count in range(n_folds):
    print(f'CV {counter}/{n_folds}')
    
    indexes = np.arange(n_folds).astype(int)    
    indexes = np.delete(indexes, obj = n_count, axis = 0) 
    indexes = np.r_[values[indexes[0]], values[indexes[1]], values[indexes[2]], values[indexes[3]]]
    
    X_train = train.loc[train['time_id'].isin(indexes), features_to_consider]
    y_train = train.loc[train['time_id'].isin(indexes), 'target']
    X_test = train.loc[train['time_id'].isin(values[n_count]), features_to_consider]
    y_test = train.loc[train['time_id'].isin(values[n_count]), 'target']

    # Model    
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006),
        loss=root_mean_squared_per_error
    )
    
    num_data = X_train[features_to_consider]
    scaler = MinMaxScaler(feature_range = (-1, 1))         
    num_data = scaler.fit_transform(num_data.to_numpy())    
    target = y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.to_numpy())

    model.fit(num_data, 
              target,               
              batch_size = 2048,
              epochs = 1000,
              validation_data = (num_data_test, y_test),
              callbacks = [es, plateau],
              validation_batch_size = len(y_test),
              shuffle = True,
              verbose = 1)

    preds = model.predict(num_data_test).reshape(1,-1)[0]
    
    score = round(rmspe(y_true = y_test, y_pred = preds),5)
    print(f'Fold {counter} {model_name}: {score}')
    scores_folds[model_name].append(score)
    
    tt = scaler.transform(test[features_to_consider].to_numpy())
    test_predictions_nn += model.predict(tt).reshape(1,-1)[0].clip(0,1e10)/n_folds
       
    counter += 1

#### Output predictions.

In [ ]:
test["row_id"] = test["stock_id"].astype(str) + "-" + test["time_id"].astype(str) 
test['target'] = 0.5 * test_predictions_nn + 0.5 * predictions_lgb

score = round(rmspe(y_true = train['target'].values, y_pred = train[pred_name].values),5)
print(f'RMSPE {model_name}: {score} - Folds: {scores_folds[model_name]}')

display(test[['row_id', 'target']].head(3))
test[['row_id', 'target']].to_csv('submission.csv',index = False)